# Lab-10-1 Relu
## sigmoid 의 단점 : Vanishing Gradient(기울이 0에 수렴하는 부분이 매우 많음)
## Relu의 장점 : max(0, x)로 양수부분에서 개선됨
## Leaky relu : Relu+ 음수부분에서 a*x (a는 엄청 작은 수)
### - 함수 목록
#### tf.keras.activations : sigmoid, tanh, relu, elu, selu
#### tf.keras.layers : leaky relu

In [24]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist ## mnist 데이터를 사용한다.
from time import time
import os
print(tf.__version__)
## tf.enable_eager_execution() : Tensorflow 2.x has eager_execution enabled by default 

2.5.0


## Load mnist

In [25]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [26]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    ## tf의 형식 : [batch_size, height, width, channel]         (tf의 형식 맞추기 위해 채널 추가)
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]   axis = -1은 배열의 맨끝 [3]에 추가하겠다는 뜻
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1] : 255로 나누면 됨!
    
    ## One hot incoding : 나중에 처리하기 편하게 데이터를 배열에 삽입
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]
    
    ## normalize된 결과, 배열에 삽입된 결과 반환
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

## Define loss

In [34]:
## 최적의 loss를 구하는 함수
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))
    return loss
##최적의 accuracy를 구하는 함수
def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy
## 최적의 기울기를 구하는 함수
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [28]:
## shape를 평쳐주는 역할
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) : ##unit : outputchannel의 수   weight initializer
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

##우리가 사용할 함수
def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

## Create network - class로 하는방법, 함수로 하는 방법이 있음

In [29]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [30]:
def create_model_function(label_dim) : ##output 개수를 input으로 한다(label_dim) : 여기는 mnist이므로 10개
    weight_init = tf.keras.initializers.RandomNormal() ## 평균0, 분산1인 랜덤 가우시안 분포 사용

    model = tf.keras.Sequential()
    model.add(flatten()) ##이미지 shape 바꾸기(나중에 connected 이용하기 위함)

    for i in range(2) : ##[N, 784] 를 두번 나누어 실행 [N, 256] *2
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init)) ## 10개의 output으로 바꾸기

    return model

## Experiments parameters

In [31]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

## Experiments models

In [32]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [33]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                print("here")
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Success to read nn_relu-936-1
 [*] Load SUCCESS
